# Update Headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

print('No external dependencies required.\n')


No external dependencies required.



In [2]:
# Some of the dependencies, to trigger the dependency checker:
# data/news-us/nyt/news-us-nyt/latest.json
# data/news-us/wsj/news-us-wsj/latest.json


from pathlib import Path
import csv
import json
import xml.etree.ElementTree as ET
from datetime import datetime, timezone, timedelta
from email.utils import parsedate_to_datetime
import shutil

BASE_DIR = Path.cwd()
REPO_DIR = BASE_DIR
while not ((REPO_DIR / 'data').exists() and (REPO_DIR / 'analysis').exists()):
    if REPO_DIR.parent == REPO_DIR:
        raise FileNotFoundError('Repository root not found')
    REPO_DIR = REPO_DIR.parent
DATA_DIR = REPO_DIR / 'data'
HEADLINES_DIR = REPO_DIR / 'analysis/headlines'
HEADLINES_DIR.mkdir(parents=True, exist_ok=True)

def parse_pubdate(date_str):
    try:
        dt = parsedate_to_datetime(date_str) if date_str else None
        if dt is None:
            return None
        if dt.tzinfo is None:
            dt = dt.replace(tzinfo=timezone.utc)
        return dt.astimezone(timezone.utc)
    except Exception:
        return None

def format_pubdate(dt):
    return dt.strftime('%Y-%m-%d-%H-%M-%S +0000') if dt else ''

def parse_feed(path: Path):
    entries = []
    if path.suffix == '.json':
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        for item in data.get('entries', []):
            title = item.get('title')
            link = item.get('link')
            pub = parse_pubdate(item.get('published'))
            if title and link:
                entries.append((pub, title.strip(), link.strip()))
    else:
        try:
            tree = ET.parse(path)
            root = tree.getroot()
        except ET.ParseError:
            return entries
        for item in root.iter():
            if item.tag.lower().endswith(('item', 'entry')):
                title = None
                link = None
                pub = None
                for child in item:
                    tag = child.tag.lower()
                    if tag.endswith('title'):
                        title = (child.text or '').strip()
                    if tag.endswith('link'):
                        link = (child.text or '').strip() or child.attrib.get('href')
                    if tag.endswith(('pubdate', 'published', 'updated')):
                        pub = parse_pubdate((child.text or '').strip())
                if title and link:
                    entries.append((pub, title, link))
    return entries

def collect_headlines():
    all_entries = []
    feed_info = {}
    for source in DATA_DIR.iterdir():
        if source.is_dir() and source.name.startswith('news'):
            candidates = [p for p in source.rglob('latest.*') if p.suffix in {'.json', '.rss', '.xml'}]
            if not candidates:
                candidates = [p for p in source.rglob('*') if p.suffix in {'.json', '.rss', '.xml'}]
            if not candidates:
                continue
            latest_file = max(candidates, key=lambda p: p.stat().st_mtime)
            source_name = latest_file.relative_to(DATA_DIR).parts[1]
            feed_entries = parse_feed(latest_file)
            if feed_entries:
                recent = max((pub for pub, _t, _l in feed_entries if pub), default=None)
                feed_info[source_name] = {'count': len(feed_entries), 'recent': recent}
            for pub, title, link in feed_entries:
                all_entries.append((pub, title, link, source_name))
    return all_entries, feed_info

def _date_key(date_str):
    try:
        return parsedate_to_datetime(date_str) if date_str else datetime.min
    except Exception:
        return datetime.min

def update_headlines():
    timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00')
    hourly_file = HEADLINES_DIR / f"{timestamp}.csv"
    if hourly_file.exists():
        print(f"{hourly_file.name} already exists. Skipping update.")
        return
    entries, feed_info = collect_headlines()
    entries.sort(key=lambda r: r[0] or datetime.min.replace(tzinfo=timezone.utc), reverse=True)
    deduped = []
    seen_titles = set()
    seen_links = set()
    for pub, title, link, src in entries:
        t_key = title.lower()
        l_key = link.lower()
        if t_key in seen_titles or l_key in seen_links:
            continue
        deduped.append((pub, src, title, link))
        seen_titles.add(t_key)
        seen_links.add(l_key)
    cutoff = datetime.now(timezone.utc) - timedelta(days=1)
    deduped = [r for r in deduped if r[0] and r[0] >= cutoff]
    with open(hourly_file, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(['pubdate', 'source', 'title', 'link'])
        for pub, src, title, link in deduped:
            writer.writerow([format_pubdate(pub), src, title, link])
    latest_file = HEADLINES_DIR / "latest.csv"
    shutil.copy(hourly_file, latest_file)
    print(f"Wrote {hourly_file} and updated latest.csv")
    print()
    print('Feed summary:')
    print(f"{'source':<20} {'count':>5}  {'most recent'}")
    for src, info in sorted(feed_info.items()):
        print(f"{src:<20} {info['count']:5}  {format_pubdate(info['recent'])}")
    # Get the most recent headline


update_headlines()







2025-07-12-05-00.csv already exists. Skipping update.


/tmp/ipykernel_2333/1894882967.py:99: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00')
